# **ANÁLISIS PREDICTIVO DE LOS DATOS**

In [62]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imblearn
from functools import reduce
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore", category=UserWarning)

In [63]:
df_final = pd.read_csv('df_final.csv')
df_final = df_final.drop(['Unnamed: 0'],axis=1)
df_final

,ehr,birth_date,diagnosis_date,death_date,er,her2,ki67,pr,pregnancy,birth,...,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,10011773,05-07-1959,02-04-2015,NaN,0.0,0.0,19.0,1.0,0.0,0.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,0,SIN TRATAMIENTO,1.0,0.0,0.0,0.0
1,10020495,02-10-1953,04-12-2017,NaN,1.0,0.0,9.0,0.0,0.0,0.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0
2,10030299,16-08-1966,27-06-2019,NaN,1.0,0.0,18.0,1.0,3.0,3.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0
3,10030824,03-03-1953,07-09-2018,NaN,1.0,0.0,18.0,1.0,0.0,0.0,...,0.0,2,2.0,0.0,IIIA,IIIA,3.0,0.0,1.0,1.0
4,10041592,06-07-1959,19-11-2018,NaN,0.0,0.0,65.0,0.0,0.0,0.0,...,0.0,4,0.0,0.0,IB,IA,2.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99343880,23-08-1944,28-05-2018,NaN,1.0,0.0,68.0,1.0,2.0,2.0,...,1.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0
11163,99443402,17-06-1959,21-01-2018,NaN,1.0,0.0,22.0,0.0,0.0,0.0,...,1.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0
11164,99656792,26-03-1976,23-07-2017,NaN,1.0,1.0,49.0,0.0,0.0,0.0,...,0.0,4,0.0,0,IA,SIN TRATAMIENTO,1.0,0.0,0.0,1.0
11165,99690760,21-07-1961,28-10-2013,NaN,1.0,0.0,20.0,1.0,1.0,1.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0


In [64]:
df_final['n_tumor'].value_counts()


n_tumor
1.0    9971
2.0    1196
Name: count, dtype: int64

In [65]:
# Filtrar las filas donde 'n_tumor' es igual a 2 en la columna 'n_tumor'
df_filtrado_n_tumor_2 = df_final[df_final['n_tumor'] == 2]

# Obtener los 'ehr' de las filas filtradas
ehr_n_tumor_2 = df_filtrado_n_tumor_2['ehr']

# Filtrar las filas donde 'ehr' está en la lista ehr_n_tumor_2 y 'n_tumor' es igual a 1
df_recaida = df_final[(df_final['ehr'].isin(ehr_n_tumor_2)) & (df_final['n_tumor'] == 1)]
df_recaida['recaida'] = 1
df_recaida

/var/folders/ly/p0f7lsjn2kq54qjv6f51ss_w0000gn/T/ipykernel_87257/2575488001.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recaida['recaida'] = 1


,ehr,birth_date,diagnosis_date,death_date,er,her2,ki67,pr,pregnancy,birth,...,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant,recaida
1,10020495,02-10-1953,04-12-2017,NaN,1.0,0.0,9.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1
17,10144916,21-10-1972,23-09-2018,NaN,1.0,0.0,20.0,1.0,0.0,0.0,...,1,0.0,1.0,IA,IA,2.0,0.0,1.0,1.0,1
18,10145135,24-07-1961,28-09-2012,28-09-2016,1.0,0.0,16.0,1.0,3.0,3.0,...,1,0.0,0.0,IB,IA,2.0,0.0,1.0,1.0,1
41,10353432,13-01-1953,15-06-2017,NaN,1.0,0.0,0.0,1.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IV,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1
43,10359877,30-06-1965,25-03-2016,09-05-2014,0.0,0.0,0.0,1.0,0.0,0.0,...,0,1.0,0.0,IIIA,IIA,2.0,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900,99343880,23-08-1944,28-05-2018,NaN,1.0,0.0,68.0,1.0,2.0,2.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0,1
9913,99443402,17-06-1959,21-01-2018,NaN,1.0,0.0,22.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0,1
9931,99656792,26-03-1976,23-07-2017,NaN,1.0,1.0,49.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,0.0,0.0,0.0,1
9933,99690760,21-07-1961,28-10-2013,NaN,1.0,0.0,20.0,1.0,1.0,1.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,1


Vamos a ver cuántos pacientes han tenido una recaída, tras hablar con los expertos del dominio (en este caso nosotros) , han considerado que tener una recaída del tumor signfica haber tenido 2 tumores, por ello, hemos obtenido aquellos pacientes que si que presentan 2 tumores...

Tras la realización, obtenemos que 1196 de los pacientes han presenciado una recaída , por ello mismo se le podría aplicar el tratamiento de los 40.000 € , que reduce así el rieso de más recaída, y el resto de los pacientes no serían de alto rieso a día de hoy
Ahora bien, cabe la posibilidad de que algunos de ellos, hayan muerto, por eso comprobamos la columna de death, si está nula, entonces a ellos se les aplicará el tratamiento mencionado, en otro caso no puesto que no han sobrevivido a la enfermedad

In [66]:
df_recaida[(df_recaida['recaida'] == 1) & (df_recaida['death_date'].isnull())]

,ehr,birth_date,diagnosis_date,death_date,er,her2,ki67,pr,pregnancy,birth,...,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant,recaida
1,10020495,02-10-1953,04-12-2017,NaN,1.0,0.0,9.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1
17,10144916,21-10-1972,23-09-2018,NaN,1.0,0.0,20.0,1.0,0.0,0.0,...,1,0.0,1.0,IA,IA,2.0,0.0,1.0,1.0,1
41,10353432,13-01-1953,15-06-2017,NaN,1.0,0.0,0.0,1.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IV,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1
44,10364469,03-07-1956,28-01-2018,NaN,1.0,0.0,19.0,0.0,0.0,0.0,...,4,0.0,0.0,IA,IA,2.0,0.0,1.0,1.0,1
54,10464026,12-07-1957,25-12-2011,NaN,0.0,0.0,50.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IV,SIN TRATAMIENTO,3.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9900,99343880,23-08-1944,28-05-2018,NaN,1.0,0.0,68.0,1.0,2.0,2.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0,1
9913,99443402,17-06-1959,21-01-2018,NaN,1.0,0.0,22.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0,1
9931,99656792,26-03-1976,23-07-2017,NaN,1.0,1.0,49.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,0.0,0.0,0.0,1
9933,99690760,21-07-1961,28-10-2013,NaN,1.0,0.0,20.0,1.0,1.0,1.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,1


De los 1196 que poseen recaídas, 1052 siguen sobreviviendo, por ello que la aplicación del tratamiento se realiza sobre ese nº de casos, ya que el resto falleció. Esto se lleva a cabo en eras de conseguir que el paciente sobreviva ya que es el objetivo del estudio.

Además, si aplicaríamos a todos los pacientes (NO FALLECIDOS), podríamos determinar cuál es el beneficio que obtendríamos en cuanto a costes... Es decir, calculamos los pacientes que tienen death a nulo, que serían los totales , * 40.000 que sería aplicar el tratamiento, pero solo aplicados a los que tienen recaída, la diferencia sería un beneficio.... si se HUBIESE APLICADO A TODOS


## **DESARROLLO DE MODELO PREDICTIVO DE RECAÍDAS**

Vamos a implementar un modelo que nos permita predecir si un paciente , dados sus datos iniciales ... es decir, los del primer tumor, va a tener o no una posible recaída.Para ello, tenemos 1196 pacientes que han poseido recaidas, y el resto NO. Para ello, incorporamos una variable objetivo, que es recaida, donde el valor 1 signifca tener recaida y un valor 0, signifca no poseer una recaída.

In [67]:
df_final

,ehr,birth_date,diagnosis_date,death_date,er,her2,ki67,pr,pregnancy,birth,...,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,10011773,05-07-1959,02-04-2015,NaN,0.0,0.0,19.0,1.0,0.0,0.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,0,SIN TRATAMIENTO,1.0,0.0,0.0,0.0
1,10020495,02-10-1953,04-12-2017,NaN,1.0,0.0,9.0,0.0,0.0,0.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0
2,10030299,16-08-1966,27-06-2019,NaN,1.0,0.0,18.0,1.0,3.0,3.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0
3,10030824,03-03-1953,07-09-2018,NaN,1.0,0.0,18.0,1.0,0.0,0.0,...,0.0,2,2.0,0.0,IIIA,IIIA,3.0,0.0,1.0,1.0
4,10041592,06-07-1959,19-11-2018,NaN,0.0,0.0,65.0,0.0,0.0,0.0,...,0.0,4,0.0,0.0,IB,IA,2.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99343880,23-08-1944,28-05-2018,NaN,1.0,0.0,68.0,1.0,2.0,2.0,...,1.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0
11163,99443402,17-06-1959,21-01-2018,NaN,1.0,0.0,22.0,0.0,0.0,0.0,...,1.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0
11164,99656792,26-03-1976,23-07-2017,NaN,1.0,1.0,49.0,0.0,0.0,0.0,...,0.0,4,0.0,0,IA,SIN TRATAMIENTO,1.0,0.0,0.0,1.0
11165,99690760,21-07-1961,28-10-2013,NaN,1.0,0.0,20.0,1.0,1.0,1.0,...,0.0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0


In [68]:
# Merge para agregar la columna 'recaida' al DataFrame df_final
df_final = pd.merge(df_final, df_recaida[['ehr', 'recaida']], how='left', on='ehr')
df_final


,ehr,birth_date,diagnosis_date,death_date,er,her2,ki67,pr,pregnancy,birth,...,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant,recaida
0,10011773,05-07-1959,02-04-2015,NaN,0.0,0.0,19.0,1.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,0,SIN TRATAMIENTO,1.0,0.0,0.0,0.0,NaN
1,10020495,02-10-1953,04-12-2017,NaN,1.0,0.0,9.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1.0
2,10030299,16-08-1966,27-06-2019,NaN,1.0,0.0,18.0,1.0,3.0,3.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,NaN
3,10030824,03-03-1953,07-09-2018,NaN,1.0,0.0,18.0,1.0,0.0,0.0,...,2,2.0,0.0,IIIA,IIIA,3.0,0.0,1.0,1.0,NaN
4,10041592,06-07-1959,19-11-2018,NaN,0.0,0.0,65.0,0.0,0.0,0.0,...,4,0.0,0.0,IB,IA,2.0,1.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99343880,23-08-1944,28-05-2018,NaN,1.0,0.0,68.0,1.0,2.0,2.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0,1.0
11163,99443402,17-06-1959,21-01-2018,NaN,1.0,0.0,22.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0,1.0
11164,99656792,26-03-1976,23-07-2017,NaN,1.0,1.0,49.0,0.0,0.0,0.0,...,4,0.0,0,IA,SIN TRATAMIENTO,1.0,0.0,0.0,1.0,1.0
11165,99690760,21-07-1961,28-10-2013,NaN,1.0,0.0,20.0,1.0,1.0,1.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,1.0


In [69]:

df_final['recaida'].fillna(0, inplace=True)
df_final

,ehr,birth_date,diagnosis_date,death_date,er,her2,ki67,pr,pregnancy,birth,...,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant,recaida
0,10011773,05-07-1959,02-04-2015,NaN,0.0,0.0,19.0,1.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,0,SIN TRATAMIENTO,1.0,0.0,0.0,0.0,0.0
1,10020495,02-10-1953,04-12-2017,NaN,1.0,0.0,9.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1.0
2,10030299,16-08-1966,27-06-2019,NaN,1.0,0.0,18.0,1.0,3.0,3.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,0.0
3,10030824,03-03-1953,07-09-2018,NaN,1.0,0.0,18.0,1.0,0.0,0.0,...,2,2.0,0.0,IIIA,IIIA,3.0,0.0,1.0,1.0,0.0
4,10041592,06-07-1959,19-11-2018,NaN,0.0,0.0,65.0,0.0,0.0,0.0,...,4,0.0,0.0,IB,IA,2.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99343880,23-08-1944,28-05-2018,NaN,1.0,0.0,68.0,1.0,2.0,2.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0,1.0
11163,99443402,17-06-1959,21-01-2018,NaN,1.0,0.0,22.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0,1.0
11164,99656792,26-03-1976,23-07-2017,NaN,1.0,1.0,49.0,0.0,0.0,0.0,...,4,0.0,0,IA,SIN TRATAMIENTO,1.0,0.0,0.0,1.0,1.0
11165,99690760,21-07-1961,28-10-2013,NaN,1.0,0.0,20.0,1.0,1.0,1.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,1.0


In [70]:
df_final['recaida'].isnull().sum()

0

In [71]:
df_final['recaida'].value_counts()

recaida
0.0    8775
1.0    2392
Name: count, dtype: int64

Antes de realizar el modelo, vemos que los datos están muy desbalanceados,por lo cuál es una consideración importante de tomar en el desarrollo del modelo.

Existen varias alternativas:
- Oversampling de la Clase Minoritaria: Aumenta la cantidad de instancias de la clase minoritaria para equilibrar las proporciones.
- Undersampling de la Clase Mayoritaria: Reduce la cantidad de instancias de la clase mayoritaria para equilibrar las proporciones.
- Ponderación de Clases: algunos algoritmos de clasificación permiten asignar pesos a las clases para dar más importancia a la clase minoritaria.

Primero de nada separamos las variables en X e Y, donde la Y es el objetivo, es decir, la recaída.

In [72]:


# Asignar 'SIN TRATAMIENTO' a las columnas específicas basado en la condición
df_final.loc[df_final['stage_after_neo'] == 'SIN TRATAMIENTO', 
                    ['t_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj']] = 'SIN TRATAMIENTO'

df_final

,ehr,birth_date,diagnosis_date,death_date,er,her2,ki67,pr,pregnancy,birth,...,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant,recaida
0,10011773,05-07-1959,02-04-2015,NaN,0.0,0.0,19.0,1.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,0,SIN TRATAMIENTO,1.0,0.0,0.0,0.0,0.0
1,10020495,02-10-1953,04-12-2017,NaN,1.0,0.0,9.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1.0
2,10030299,16-08-1966,27-06-2019,NaN,1.0,0.0,18.0,1.0,3.0,3.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,0.0
3,10030824,03-03-1953,07-09-2018,NaN,1.0,0.0,18.0,1.0,0.0,0.0,...,2,2.0,0.0,IIIA,IIIA,3.0,0.0,1.0,1.0,0.0
4,10041592,06-07-1959,19-11-2018,NaN,0.0,0.0,65.0,0.0,0.0,0.0,...,4,0.0,0.0,IB,IA,2.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99343880,23-08-1944,28-05-2018,NaN,1.0,0.0,68.0,1.0,2.0,2.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,2.0,0.0,0.0,0.0,1.0
11163,99443402,17-06-1959,21-01-2018,NaN,1.0,0.0,22.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IIB,SIN TRATAMIENTO,3.0,0.0,1.0,0.0,1.0
11164,99656792,26-03-1976,23-07-2017,NaN,1.0,1.0,49.0,0.0,0.0,0.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,0.0,0.0,1.0,1.0
11165,99690760,21-07-1961,28-10-2013,NaN,1.0,0.0,20.0,1.0,1.0,1.0,...,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,IA,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,1.0


In [73]:
columnas_seleccionadas = [
    'ehr','n_tumor', 't_category', 'n_category', 'm_category', 'stage_diagnosis',
    't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo',
    'grade', 'ductal', 'lobular', 'neoadjuvant',
    'er', 'pr', 'her2', 'ki67','recaida'
]

df_modelo = df_final[columnas_seleccionadas]
df_modelo


,ehr,n_tumor,t_category,n_category,m_category,stage_diagnosis,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_after_neo,grade,ductal,lobular,neoadjuvant,er,pr,her2,ki67,recaida
0,10011773,1.0,IS,0.0,0.0,0,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,1.0,0.0,0.0,0.0,0.0,1.0,0.0,19.0,0.0
1,10020495,1.0,1,0.0,0.0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,2.0,1.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0
2,10030299,1.0,1,0.0,0.0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,1.0,1.0,0.0,18.0,0.0
3,10030824,1.0,2,1.0,0.0,IIIA,2,2.0,0.0,IIIA,3.0,0.0,1.0,1.0,1.0,1.0,0.0,18.0,0.0
4,10041592,1.0,1,1.0,0.0,IB,4,0.0,0.0,IA,2.0,1.0,0.0,1.0,0.0,0.0,0.0,65.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99343880,2.0,3,1.0,1.0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,2.0,0.0,0.0,0.0,1.0,1.0,0.0,68.0,1.0
11163,99443402,2.0,2,1.0,1.0,IIB,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,3.0,0.0,1.0,0.0,1.0,0.0,0.0,22.0,1.0
11164,99656792,2.0,2,0.0,0.0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,1.0,0.0,0.0,1.0,1.0,0.0,1.0,49.0,1.0
11165,99690760,2.0,3,0.0,0.0,IA,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,SIN TRATAMIENTO,1.0,1.0,0.0,0.0,1.0,1.0,0.0,20.0,1.0


In [74]:
columnas_seleccionadas_2 = [
    't_category', 'n_category', 'm_category', 'stage_diagnosis',
    't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo',
    'grade', 'ductal', 'lobular', 'neoadjuvant',
    'er', 'pr', 'her2', 'ki67'
]
X = df_modelo[columnas_seleccionadas_2]
y = df_modelo['recaida']

In [75]:
print('Original dataset shape %s' % Counter(y))
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({0.0: 8775, 1.0: 2392})
Resampled dataset shape Counter({0.0: 2392, 1.0: 2392})


In [76]:
X_res.dtypes

t_category                  object
n_category                 float64
m_category                 float64
stage_diagnosis             object
t_category_after_neoadj     object
n_category_after_neoadj     object
m_category_after_neoadj     object
stage_after_neo             object
grade                      float64
ductal                     float64
lobular                    float64
neoadjuvant                float64
er                         float64
pr                         float64
her2                       float64
ki67                       float64
dtype: object

In [77]:
# Suponiendo que 'df_seleccionado' es tu DataFrame

# Lista de columnas que deseas convertir a tipo 'object' excepto 'ki67'
columnas_object = ['t_category', 'n_category', 'm_category', 'stage_diagnosis',
                    't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo',
                    'grade', 'ductal', 'lobular', 'neoadjuvant', 'er', 'pr', 'her2','ki67']

# Convertir las columnas a tipo 'object' excepto 'ki67'
X_res[columnas_object] = X_res[columnas_object].astype(str)


In [78]:
X_res.dtypes

t_category                 object
n_category                 object
m_category                 object
stage_diagnosis            object
t_category_after_neoadj    object
n_category_after_neoadj    object
m_category_after_neoadj    object
stage_after_neo            object
grade                      object
ductal                     object
lobular                    object
neoadjuvant                object
er                         object
pr                         object
her2                       object
ki67                       object
dtype: object

In [79]:
for columna in X_res.columns:
    valores_unicos = X_res[columna].unique()
    print(f'Valores únicos en la columna {columna}:\n{valores_unicos}\n')


Valores únicos en la columna t_category:
['0' '1' '3' 'IS' '2' '4']

Valores únicos en la columna n_category:
['1.0' '0.0' '2.0' '3.0']

Valores únicos en la columna m_category:
['1.0' '0.0']

Valores únicos en la columna stage_diagnosis:
['IB' 'IA' 'IIIA' 'IV' '0' 'IIIC' 'IIA' 'IIB' 'IIIB']

Valores únicos en la columna t_category_after_neoadj:
['SIN TRATAMIENTO' '3' '0' 'IS' '2' '1' '4']

Valores únicos en la columna n_category_after_neoadj:
['SIN TRATAMIENTO' '0.0' '1.0' '3.0' '2.0']

Valores únicos en la columna m_category_after_neoadj:
['SIN TRATAMIENTO' '0.0' '1.0']

Valores únicos en la columna stage_after_neo:
['SIN TRATAMIENTO' 'IA' 'IB' '0' 'IIA' 'IIB' 'IIIA' 'IIIB' 'IV' 'IIIC']

Valores únicos en la columna grade:
['2.0' '1.0' '3.0']

Valores únicos en la columna ductal:
['1.0' '0.0']

Valores únicos en la columna lobular:
['0.0' '1.0']

Valores únicos en la columna neoadjuvant:
['0.0' '1.0']

Valores únicos en la columna er:
['1.0' '0.0']

Valores únicos en la columna pr:
[

In [80]:
# Suponiendo que 'columna' es el nombre de la columna que te interesa
columna = 'ductal'

# Itera sobre los valores de la columna e imprime el tipo de datos para cada valor
for valor in X_res[columna]:
    print(f'Tipo de datos para el valor {valor}: {type(valor)}')

Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 1.0: <class 'str'>
Tipo de datos para el valor 0.0: <class 'str'>
Tipo de datos

In [81]:
X_res[columna] = X_res[columna].astype(str)

In [82]:
# Suponiendo que 'X_res' es tu DataFrame
label_encoder = LabelEncoder()

# Itera sobre todas las columnas y aplica el LabelEncoder
for columna in X_res.columns:
    X_res[columna] = label_encoder.fit_transform(X_res[columna])

# Ahora, todas las columnas en X_res contienen valores numéricos después de la codificación

In [83]:
X_res

,t_category,n_category,m_category,stage_diagnosis,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_after_neo,grade,ductal,lobular,neoadjuvant,er,pr,her2,ki67
9611,0,1,1,2,6,4,2,9,1,1,0,0,1,1,0,8
6742,0,1,0,1,3,0,0,1,0,1,0,1,0,1,0,10
3858,1,0,0,1,6,4,2,9,1,1,0,0,1,1,0,15
7397,0,2,0,5,0,1,0,2,0,1,0,1,1,1,0,19
1159,3,0,0,8,5,0,0,0,1,0,0,1,1,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,3,1,1,1,6,4,2,9,1,0,0,0,1,1,0,66
11163,2,1,1,4,6,4,2,9,2,0,1,0,1,0,0,16
11164,2,0,0,1,6,4,2,9,0,0,0,1,1,0,1,45
11165,3,0,0,1,6,4,2,9,0,1,0,0,1,1,0,14


In [84]:
y_res

9611     0.0
6742     0.0
3858     0.0
7397     0.0
1159     0.0
        ... 
11162    1.0
11163    1.0
11164    1.0
11165    1.0
11166    1.0
Name: recaida, Length: 4784, dtype: float64

In [85]:
# Suponiendo que 'X_res' es tu DataFrame con las variables ya transformadas por LabelEncoder
# y 'y' es tu variable objetivo

# Inicializa el modelo de árbol de decisión
modelo_arbol = DecisionTreeClassifier()

# Realiza la validación cruzada y obtén las puntuaciones de precisión
scores = cross_val_score(modelo_arbol, X_res, y_res, cv=10, scoring='accuracy')

In [86]:
# Muestra las puntuaciones de precisión para cada pliegue
print(f'Puntuaciones de precisión por pliegue: {scores}')

# Muestra la precisión promedio
print(f'Precisión promedio: {scores.mean()}')

Puntuaciones de precisión por pliegue: [0.50730689 0.46764092 0.51983299 0.47599165 0.4916318  0.70292887
 0.70292887 0.70711297 0.66108787 0.71129707]
Precisión promedio: 0.5947759890287472


In [87]:
# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)
# Listamos los modelos a evaluar
modelos = [
    ('Random Forest', RandomForestClassifier()),
    ('SVM', SVC()),
    ('Logistic Regression', LogisticRegression()),
    ('KNN', KNeighborsClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Decision Tree', DecisionTreeClassifier())
]

# Diccionario para almacenar los resultados de la validación cruzada
resultados = {}

In [88]:
# Itera sobre cada modelo
for nombre, modelo in modelos:
    # Realiza la validación cruzada y obtén las puntuaciones de precisión
    scores = cross_val_score(modelo, X_train, y_train, cv=10, scoring='accuracy')
    
    # Almacena las puntuaciones en el diccionario de resultados
    resultados[nombre] = scores.mean()

# Encuentra el modelo con el mejor rendimiento
mejor_modelo = max(resultados, key=resultados.get)
print(f'Mejor modelo: {mejor_modelo} con precisión media: {resultados[mejor_modelo]}')

# Entrena el mejor modelo en el conjunto de entrenamiento completo
modelo_final = next(modelo for nombre, modelo in modelos if nombre == mejor_modelo)
modelo_final.fit(X_train, y_train)

# Evalúa el rendimiento en el conjunto de prueba
predicciones = modelo_final.predict(X_test)
precision_test = accuracy_score(y_test, predicciones)
print(f'Precisión en el conjunto de prueba: {precision_test}')

Mejor modelo: Gradient Boosting con precisión media: 0.6908616187989557
Precisión en el conjunto de prueba: 0.7063740856844305


In [89]:
resultados

{'Random Forest': 0.6670895246948179,
 'SVM': 0.6095847060270939,
 'Logistic Regression': 0.6467014339808348,
 'KNN': 0.6106564324087871,
 'Gradient Boosting': 0.6908616187989557,
 'Decision Tree': 0.6015269366943257}